In [94]:
import json
import dash
from dash import dcc, html, Input, Output, dash_table
import dash_bootstrap_components as dbc
import dash_leaflet as dl
import pandas as pd
import geopandas as gpd
from shapely import wkt
from pyproj import Transformer
from shapely.geometry import LineString, MultiLineString

# Data imports
vz_moped = pd.read_csv('../Output/annualized_statistics.csv')

# vz_moped = pd.read_csv('https://data.austintexas.gov/resource/a65x-x4y7.csv?$limit=9999999')

# Dropdown menu formatting
columns_to_display = [
    "component_name",
    "component_subtype",
    "component_work_types",
    "type_name",
    "substantial_completion_date",
    "pre_annualized_crash_rate",
    "post_annualized_crash_rate",
    "delta_crash_rate",
    "pre_annualized_fatal_crash_rate",
    "post_annualized_fatal_crash_rate",
    "delta_fatal_crash_rate",
    "pre_annualized_injury_rate",
    "post_annualized_injury_rate",
    "delta_injury_rate",
    "pre_annualized_death_rate",
    "post_annualized_death_rate",
    "delta_death_rate",
    "pre_annualized_cost",
    "post_annualized_cost",
    "delta_comp_cost",
    "component_had_fatal_crash",
]

column_labels = {    
    "moped_component_id": "Component ID",
    "component_name": "Name",
    "component_name_full": "Full Name",
    "component_subtype": "Subtype",
    "component_work_types": "Work Types",
    "type_name": "Type",
    "substantial_completion_date": "Completion Date",
    "pre_annualized_crash_rate": "Pre Annualized Crash Rate",
    "post_annualized_crash_rate": "Post Annualized Crash Rate",
    "delta_crash_rate": "Delta Crash Rate",
    "pre_annualized_fatal_crash_rate": "Pre Annualized Fatal Crash Rate",
    "post_annualized_fatal_crash_rate": "Post Annualized Fatal Crash Rate",
    "delta_fatal_crash_rate": "Delta Fatal Crash Rate",
    "pre_annualized_injury_rate": "Pre Annualized Injury Rate",
    "post_annualized_injury_rate": "Post Annualized Injury Rate",
    "delta_injury_rate": "Delta Injury Rate",
    "pre_annualized_death_rate": "Pre Annualized Death Rate",
    "post_annualized_death_rate": "Post Annualized Death Rate",
    "delta_death_rate": "Delta Death Rate",
    "pre_annualized_cost": "Pre Annualized Composite Cost",
    "post_annualized_cost": "Post Annualized Composite Cost",
    "delta_comp_cost": "Delta Composite Cost",
    "component_had_fatal_crash": "Had Fatal Crash",
}

# Data manipulation

# Renaming columns
vz_moped.rename(
    columns={
        'pre_annualized_fatal_crash': 'pre_annualized_fatal_crash_rate',
        'post_annualized_fatal_crash': 'post_annualized_fatal_crash_rate'
    },
    inplace=True
)

# Rounding columns which will be displayed on the table
columns_to_round = [
    "pre_annualized_crash_rate",
    "post_annualized_crash_rate",
    "delta_crash_rate",
    "pre_annualized_fatal_crash_rate",
    "post_annualized_fatal_crash_rate",
    "delta_fatal_crash_rate",
    "pre_annualized_injury_rate",
    "post_annualized_injury_rate",
    "delta_injury_rate",
    "pre_annualized_death_rate",
    "post_annualized_death_rate",
    "delta_death_rate",
    "pre_annualized_cost",
    "post_annualized_cost",
    "delta_comp_cost"
]

DECIMALS = 2    
vz_moped[columns_to_round] = vz_moped[columns_to_round].apply(lambda x: round(x, DECIMALS))

# Changing null values to "N/As"
vz_moped["component_subtype"] = vz_moped["component_subtype"].fillna("N/A")
vz_moped["component_work_types"] = vz_moped["component_work_types"].fillna("N/A")

# Converting to datetime
vz_moped["substantial_completion_date"] = pd.to_datetime(
    vz_moped["substantial_completion_date"]
)

# Creating completion year variable
vz_moped["completion_year"] = vz_moped["substantial_completion_date"].dt.year

# Keeping only the date in the substantial completition date column
vz_moped["substantial_completion_date"] = vz_moped["substantial_completion_date"].dt.date

# Flag for involving fatality
# 1 if pre/post fatal crash rate is not null
vz_moped["component_had_fatal_crash"] = (
    vz_moped["pre_annualized_fatal_crash_rate"] > 0
) | (vz_moped["post_annualized_fatal_crash_rate"] > 0)
vz_moped["component_had_fatal_crash"] = vz_moped["component_had_fatal_crash"].apply(
    lambda x: "Yes" if x else "No"
)

# Setting the geometry column
vz_moped["line_geometry"] = vz_moped["line_geometry"].apply(wkt.loads)

transformer = Transformer.from_crs("EPSG:32614", "EPSG:4326", always_xy=True)

# Function to transform coordinates using a transformer
def transform_coordinates(geom, transformer):
    if geom.geom_type == "Point":
        x, y = geom.x, geom.y
        return Point(*transformer.transform(x, y)[::-1])
    elif geom.geom_type == "LineString":
        return LineString([transformer.transform(x, y)[::-1] for x, y in geom.coords])
    elif geom.geom_type == "MultiLineString":
        return MultiLineString(
            [
                LineString([transformer.transform(x, y)[::-1] for x, y in line.coords])
                for line in geom.geoms
            ]
        )
    else:
        raise ValueError(f"Unsupported geometry type: {geom.geom_type}")


# Function to create Geojson data
def create_geojson(filtered_df):
    if filtered_df.empty:
        return {}

    gdf = gpd.GeoDataFrame(filtered_df, geometry="line_geometry")

    # Convert datetime columns to string for geojson export
    for column in gdf.select_dtypes(include=["datetimetz", "datetime64"]).columns:
        gdf[column] = gdf[column].astype(str)

    # Set up the transformer
    transformer = Transformer.from_crs("epsg:32614", "epsg:4326", always_xy=True)

    # Transform the coordinates
    gdf["line_geometry"] = gdf["line_geometry"].apply(
        lambda geom: transform_coordinates(geom, transformer)
    )

    # Get GeoJSON data
    geojson_data = (gdf.__geo_interface__)

    # print(f"\n\nGeoJSON data:\n{json.dumps(geojson_data, indent=4)}")

    # Add the necessary properties for tooltip
    for feature in geojson_data['features']:
        properties = feature['properties']
        tooltip_content = f"""
            <b>Component Name:</b> {properties.get('component_name', 'N/A')}<br>
            <b>Completion Year:</b> {properties.get('completion_year', 'N/A')}<br>
            <b>Component Subtype:</b> {properties.get('component_subtype', 'N/A')}<br>
            <b>Work Type:</b> {properties.get('component_work_types', 'N/A')}<br>
            <b>Fatal Crash:</b> {properties.get('component_had_fatal_crash', 'N/A')}<br>
            <b>Delta Composite Cost:</b> {properties.get('delta_comp_cost', 'N/A')}
        """
        feature['properties']['tooltip'] = tooltip_content

    # Flipping the coordinates
    geojson_data = flip_coordinates(geojson_data)

    return geojson_data


def flip_coordinates(geojson):
    # Parse the GeoJSON string into a Python dictionary

    def flip_coords(coords):
        # Flip the lat and long for a given coordinate pair
        return [coords[1], coords[0]]

    def process_geometry(geometry):
        if geometry["type"] == "Point":
            geometry["coordinates"] = flip_coords(geometry["coordinates"])
        elif geometry["type"] == "LineString" or geometry["type"] == "MultiPoint":
            geometry["coordinates"] = [
                flip_coords(coord) for coord in geometry["coordinates"]
            ]
        elif geometry["type"] == "Polygon" or geometry["type"] == "MultiLineString":
            geometry["coordinates"] = [
                [flip_coords(coord) for coord in ring]
                for ring in geometry["coordinates"]
            ]
        elif geometry["type"] == "MultiPolygon":
            geometry["coordinates"] = [
                [[flip_coords(coord) for coord in ring] for ring in polygon]
                for polygon in geometry["coordinates"]
            ]
        elif geometry["type"] == "GeometryCollection":
            for geom in geometry["geometries"]:
                process_geometry(geom)
        return geometry

    def process_feature(feature):
        feature["geometry"] = process_geometry(feature["geometry"])
        return feature

    if geojson["type"] == "FeatureCollection":
        geojson["features"] = [
            process_feature(feature) for feature in geojson["features"]
        ]
    elif geojson["type"] == "Feature":
        geojson["geometry"] = process_geometry(geojson["geometry"])
    else:  # For direct geometries
        geojson = process_geometry(geojson)

    # Convert the modified dictionary back to a JSON string
    return geojson


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])


def create_card(title, dropdown_id, options):
    return dbc.Card(
        [
            dbc.CardHeader(title, className="card-header"),
            dbc.CardBody(
                dcc.Dropdown(
                    id=dropdown_id,
                    options=[{"label": i, "value": i} for i in options],
                    multi=True
                ), className="card=body"
            ),
        ]
    )


app.layout = dbc.Container(
    [
        dbc.Row(
            dbc.Col(html.H1("VisionZero Moped Pre/Post Analysis", className="app_title"), width=12),
            className="justify-content-center",
        ),
        dbc.Row(
            [
                dbc.Col(
                    create_card(
                        "Completion year",
                        "dropdown_year",
                        sorted(vz_moped["completion_year"].unique()),
                    ),
                    className="navigation_row_col_style",
                ),
                dbc.Col(
                    create_card(
                        "Component name",
                        "dropdown_component_name",
                        vz_moped["component_name"].unique(),
                    ),
                    className="navigation_row_col_style",
                ),
                dbc.Col(
                    create_card(
                        "Component sub-type",
                        "dropdown_component_subtype",
                        vz_moped["component_subtype"].unique(),
                    ),
                    className="navigation_row_col_style",
                ),
                dbc.Col(
                    create_card(
                        "Component work-type",
                        "dropdown_work_type",
                        vz_moped["component_work_types"].unique(),
                    ),
                    className="navigation_row_col_style",
                ),
                dbc.Col(
                    create_card(
                        "Fatal crash",
                        "dropdown_fatal_crash",
                        vz_moped["component_had_fatal_crash"].unique(),
                    ),
                    className="navigation_row_col_style",
                ),
            ],
            className="navigation_row",
        ),
        dbc.Row(
            [
                dbc.Col(
                    dash_table.DataTable(
                        id='data_table',
                        columns=[{'name': column_labels[col], 'id': col} for col in columns_to_display],
                        data=vz_moped[columns_to_display].to_dict('records'),
                        selected_row_ids=[],
                        style_table={'overflowX': 'auto', 'maxHeight': '600px'},  # Enable horizontal scrolling
                        style_cell={'minWidth': '120px', 'maxWidth': '200px', 'whiteSpace': 'normal', 'font-size': '13px', 'padding': '5px', 'font-family': 'Arial, sans-serif'},  # Match style with external CSS
                        style_header={
                            'fontWeight': 'bold',
                            'backgroundColor': 'lightgrey',
                            'textAlign': 'center'
                        },
                        style_data_conditional=[
                            {
                                'if': {'column_type': 'text'},
                                'textAlign': 'center'
                            },
                        ],
                        cell_selectable=True,
                        sort_action='native',
                        style_as_list_view=True,
                    ),
                    width=8
                ),
                dbc.Col(
                    dl.Map(
                        center=[30.2672, -97.7431],
                        zoom=12,
                        children=[
                            dl.TileLayer(),
                            dl.ScaleControl(
                                position="bottomleft",
                                imperial=True,
                                metric=False
                            ),
                            dl.GestureHandling(),
                            dl.GeoJSON(
                                id="geojson",
                                options=dict(style={"color": "blue", "weight": 2}),
                                hoverStyle=dict(weight=5, color='#666', dashArray=''),
                                children=[
                                    dl.Tooltip(id="geojson_tooltip", direction='top', permanent=True, className='tooltip')
                                ]
                            ),
                        ],
                        style={"width": "100%", "height": "635px"},
                        className="map",
                        attributionControl=False,
                    ),
                    width=4,
                )
            ]
        ),
    ],
    fluid=True,
    className="custom_container",
)


@app.callback(
    Output('data_table', 'data'),
    Output("geojson", "data"),
    [
        Input("dropdown_year", "value"),
        Input("dropdown_component_name", "value"),
        Input("dropdown_component_subtype", "value"),
        Input("dropdown_work_type", "value"),
        Input("dropdown_fatal_crash", "value"),
    ],
)
def update_plot(
    dropdown_year,
    dropdown_component_name,
    dropdown_component_subtype,
    dropdown_work_type,
    dropdown_fatal_crash,
):
    # Base DataFrame
    filtered_df = vz_moped

    # Convert single values to list to avoid type errors
    dropdown_year = (
        [dropdown_year]
        if dropdown_year and not isinstance(dropdown_year, list)
        else dropdown_year
    )
    dropdown_component_name = (
        [dropdown_component_name]
        if dropdown_component_name and not isinstance(dropdown_component_name, list)
        else dropdown_component_name
    )
    dropdown_component_subtype = (
        [dropdown_component_subtype]
        if dropdown_component_subtype
        and not isinstance(dropdown_component_subtype, list)
        else dropdown_component_subtype
    )
    dropdown_work_type = (
        [dropdown_work_type]
        if dropdown_work_type and not isinstance(dropdown_work_type, list)
        else dropdown_work_type
    )
    dropdown_fatal_crash = (
        [dropdown_fatal_crash]
        if dropdown_fatal_crash and not isinstance(dropdown_fatal_crash, list)
        else dropdown_fatal_crash
    )

    # Apply filters based on dropdown selection
    if dropdown_year:
        filtered_df = filtered_df[filtered_df["completion_year"].isin(dropdown_year)]
    if dropdown_component_name:
        filtered_df = filtered_df[
            filtered_df["component_name"].isin(dropdown_component_name)
        ]
    if dropdown_component_subtype:
        filtered_df = filtered_df[
            filtered_df["component_subtype"].isin(dropdown_component_subtype)
        ]
    if dropdown_work_type:
        filtered_df = filtered_df[
            filtered_df["component_work_types"].isin(dropdown_work_type)
        ]
    if dropdown_fatal_crash:
        filtered_df = filtered_df[
            filtered_df["component_had_fatal_crash"].isin(dropdown_fatal_crash)
        ]

    # Create Geojson data
    geojson_data = create_geojson(filtered_df)

    return filtered_df[columns_to_display].to_dict('records'), geojson_data


if __name__ == "__main__":
    app.run_server(debug=True, port=8222)

In [95]:
vz_moped.head()

,moped_component_id,component_work_types,type_name,line_geometry,project_name,project_id,project_component_id,substantial_completion_date,pre_annualized_crash_rate,post_annualized_crash_rate,...,post_annualized_death_rate,delta_death_rate,pre_annualized_cost,post_annualized_cost,delta_comp_cost,component_name,component_name_full,component_subtype,completion_year,component_had_fatal_crash
0,2,Modification,Signal - Mod,LINESTRING (621756.8366993731 3349129.94356965...,East 7th Street & East 8th Street / I-35,12,183.0,2022-10-10,0.12,NaN,...,NaN,NaN,17412.54,NaN,NaN,Signal,Signal - Traffic,Traffic,2022,No
1,3,Modification,Signal - Mod,LINESTRING (621800.8777623806 3349002.52492784...,East 7th Street & East 8th Street / I-35,12,182.0,2022-10-10,0.17,NaN,...,NaN,NaN,156960.69,NaN,NaN,Signal,Signal - Traffic,Traffic,2022,No
2,4,N/A,Signal - Mod,MULTILINESTRING ((621756.1558980751 3349130.60...,East 7th Street & East 8th Street / I-35,12,469.0,2022-10-10,0.16,0.74,...,0.0,0.0,45437.83,171651.23,126213.40,Intersection,Intersection - Improvement,Improvement,2022,No
3,5,Modification,Signal - Mod,LINESTRING (621722.7430791555 3349029.36478667...,East 7th Street & East 8th Street / I-35,12,16.0,2022-10-10,0.16,0.74,...,0.0,0.0,30583.35,171651.23,141067.88,Signal,Signal - Traffic,Traffic,2022,No
4,7,Modification,Signal - Mod,LINESTRING (617652.5950659687 3350801.66808639...,AMD EXPOSITION / LAKE AUSTIN SIGNAL MODS INSPE...,27,31.0,2024-02-20,0.14,NaN,...,NaN,NaN,31988.10,NaN,NaN,Signal,Signal - Traffic,Traffic,2024,No


In [96]:
vz_moped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2096 entries, 0 to 2095
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   moped_component_id                2096 non-null   int64  
 1   component_work_types              2096 non-null   object 
 2   type_name                         69 non-null     object 
 3   line_geometry                     2096 non-null   object 
 4   project_name                      2096 non-null   object 
 5   project_id                        2096 non-null   int64  
 6   project_component_id              2096 non-null   float64
 7   substantial_completion_date       2096 non-null   object 
 8   pre_annualized_crash_rate         1892 non-null   float64
 9   post_annualized_crash_rate        1472 non-null   float64
 10  delta_crash_rate                  1268 non-null   float64
 11  pre_annualized_fatal_crash_rate   1892 non-null   float64
 12  post_a